In [1]:
import pandas as pd
import requests


In [52]:
df = pd.read_parquet(r'../data/pronostico_erv/limpieza.parquet')
df.rename(columns={'CODIGO':'id_infotecnica','ESTACION':'nombre_central'}, inplace=True)
df

,id_infotecnica,nombre_central,DATE_DESDE,DATE_HASTA,P50
0,223,eolica-punta-colorada,23-01-2020 11:00,23-01-2020 12:00,1.70
1,223,eolica-punta-colorada,23-01-2020 10:00,23-01-2020 11:00,0.89
2,223,eolica-punta-colorada,23-01-2020 09:00,23-01-2020 10:00,0.34
3,223,eolica-punta-colorada,23-01-2020 08:00,23-01-2020 09:00,0.21
4,223,eolica-punta-colorada,23-01-2020 07:00,23-01-2020 08:00,0.18
...,...,...,...,...,...
9710058,319,luz-del-norte,30-11-2019 23:00,01-12-2019 00:00,0.00
9710059,319,luz-del-norte,23-11-2019 07:00,23-11-2019 08:00,129.76
9710060,319,luz-del-norte,23-11-2019 08:00,23-11-2019 09:00,129.76
9710061,319,luz-del-norte,23-11-2019 09:00,23-11-2019 10:00,129.76


In [53]:
# Check and remove duplicates, keeping the last occurrence
df = df.drop_duplicates(subset=['id_infotecnica', 'DATE_DESDE'], keep='last').copy()

In [54]:
df.DATE_DESDE = df.DATE_DESDE.str.replace('/', '-').copy()  
df.DATE_DESDE = df.DATE_DESDE.str.replace('2121', '2021').copy()
df.DATE_DESDE = df.DATE_DESDE.str.replace('#', '').copy()
df

,id_infotecnica,nombre_central,DATE_DESDE,DATE_HASTA,P50
1,223,eolica-punta-colorada,23-01-2020 10:00,23-01-2020 11:00,0.89
2,223,eolica-punta-colorada,23-01-2020 09:00,23-01-2020 10:00,0.34
3,223,eolica-punta-colorada,23-01-2020 08:00,23-01-2020 09:00,0.21
12,247,solar-llano-de-llampos,15-01-2020 04:00,15-01-2020 05:00,0.00
15,247,solar-llano-de-llampos,15-01-2020 01:00,15-01-2020 02:00,0.00
...,...,...,...,...,...
9710058,319,luz-del-norte,30-11-2019 23:00,01-12-2019 00:00,0.00
9710059,319,luz-del-norte,23-11-2019 07:00,23-11-2019 08:00,129.76
9710060,319,luz-del-norte,23-11-2019 08:00,23-11-2019 09:00,129.76
9710061,319,luz-del-norte,23-11-2019 09:00,23-11-2019 10:00,129.76


In [55]:
df = df[df['id_infotecnica']!="None"].copy()
df.id_infotecnica = df.id_infotecnica.astype(int)

In [56]:
# First try with dd/mm/yyyy format
df['datetime'] = pd.to_datetime(df.DATE_DESDE, format="%d-%m-%Y %H:%M", errors='coerce')

# Fill NaT values with second format attempt
mask = df['datetime'].isna()
df.loc[mask, 'datetime'] = pd.to_datetime(df.loc[mask, 'DATE_DESDE'], format="%Y-%m-%d %H:%M", errors='coerce')



In [57]:
df.dropna(inplace=True)
df.sort_values(by='datetime', inplace=True)

In [58]:
df

,id_infotecnica,nombre_central,DATE_DESDE,DATE_HASTA,P50,datetime
2046893,248,solar-san-andres,23-08-2016 18:00,23-08-2016 19:00,14.60,2016-08-23 18:00:00
2046898,248,solar-san-andres,24-08-2016 13:00,24-08-2016 14:00,22.00,2016-08-24 13:00:00
2046897,248,solar-san-andres,24-08-2016 14:00,24-08-2016 15:00,22.00,2016-08-24 14:00:00
2046896,248,solar-san-andres,24-08-2016 15:00,24-08-2016 16:00,22.00,2016-08-24 15:00:00
2046895,248,solar-san-andres,24-08-2016 16:00,24-08-2016 17:00,22.00,2016-08-24 16:00:00
...,...,...,...,...,...,...
923422,332,parque-pampa-solar-norte,02-07-2022 15:00,02-07-2022 16:00,40.21,2022-07-02 15:00:00
23281,1862,pfv-inca-de-oro-en,02-07-2022 15:00,02-07-2022 16:00,132.83,2022-07-02 15:00:00
923421,332,parque-pampa-solar-norte,02-07-2022 16:00,02-07-2022 17:00,33.80,2022-07-02 16:00:00
23280,1862,pfv-inca-de-oro-en,02-07-2022 16:00,02-07-2022 17:00,104.96,2022-07-02 16:00:00


In [59]:
rename_cols = {'P50':'pronostico'}
df = df.rename(columns=rename_cols)
df.drop(columns=['DATE_DESDE','DATE_HASTA'], inplace=True)
df.head()

,id_infotecnica,nombre_central,pronostico,datetime
2046893,248,solar-san-andres,14.6,2016-08-23 18:00:00
2046898,248,solar-san-andres,22.0,2016-08-24 13:00:00
2046897,248,solar-san-andres,22.0,2016-08-24 14:00:00
2046896,248,solar-san-andres,22.0,2016-08-24 15:00:00
2046895,248,solar-san-andres,22.0,2016-08-24 16:00:00


In [60]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1789186 entries, 2046893 to 252463
Data columns (total 4 columns):
 #   Column          Dtype         
---  ------          -----         
 0   id_infotecnica  int64         
 1   nombre_central  object        
 2   pronostico      float64       
 3   datetime        datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 68.3+ MB


In [61]:
df[df['id_infotecnica']==1704].describe()


,id_infotecnica,pronostico,datetime
count,5418.0,5418.000000,5418
mean,1704.0,15.242342,2021-10-17 07:46:36.677741056
min,1704.0,0.000000,2021-02-12 12:00:00
25%,1704.0,0.000000,2021-06-04 00:15:00
50%,1704.0,0.000000,2021-10-12 20:30:00
75%,1704.0,17.517500,2022-03-07 16:45:00
max,1704.0,306.130000,2022-06-02 11:00:00
std,0.0,29.202761,NaN


In [62]:
centrales = requests.get('https://api-infotecnica.coordinador.cl/v1/centrales-sin-filtrar/extended/?format=json')
data = centrales.json()
df_sen = pd.DataFrame(data)
df_sen = df_sen[['id','nombre','tipo_central_nombre','region_nombre','latitud','longitud','potencia_maxima']]
df_sen.rename(columns={'id':'id_infotecnica'}, inplace=True)
df_sen.head()

,id_infotecnica,nombre,tipo_central_nombre,region_nombre,latitud,longitud,potencia_maxima
0,110,TER CENIZAS,Termoeléctricas,Atacama,-27.484321,-70.391944,15.2545
1,145,PE MONTE REDONDO,Eólicas,Coquimbo,-31.077327,-71.657829,48.0000
2,221,PMGD HP MUCHI,Hidroeléctricas,Los Ríos,-40.316774,-72.227259,1.0000
3,236,TER CAMPICHE,Termoeléctricas,Valparaíso,-32.748294,-71.483299,269.6600
4,260,TER BOCAMINA II [No_Mostrar],Termoeléctricas,Biobío,-37.021485,-73.167313,349.5500


In [63]:
df_merged = pd.merge(df, df_sen, on='id_infotecnica', how='left')

df_merged.head()


,id_infotecnica,nombre_central,pronostico,datetime,nombre,tipo_central_nombre,region_nombre,latitud,longitud,potencia_maxima
0,248,solar-san-andres,14.6,2016-08-23 18:00:00,PFV SAN ANDRES,Solares,Atacama,-27.254224,-70.111221,50.6
1,248,solar-san-andres,22.0,2016-08-24 13:00:00,PFV SAN ANDRES,Solares,Atacama,-27.254224,-70.111221,50.6
2,248,solar-san-andres,22.0,2016-08-24 14:00:00,PFV SAN ANDRES,Solares,Atacama,-27.254224,-70.111221,50.6
3,248,solar-san-andres,22.0,2016-08-24 15:00:00,PFV SAN ANDRES,Solares,Atacama,-27.254224,-70.111221,50.6
4,248,solar-san-andres,22.0,2016-08-24 16:00:00,PFV SAN ANDRES,Solares,Atacama,-27.254224,-70.111221,50.6


In [64]:
# Check for duplicates in id_infotecnica and datetime columns
df_merged[df_merged.duplicated(['id_infotecnica', 'datetime'], keep=False)]


,id_infotecnica,nombre_central,pronostico,datetime,nombre,tipo_central_nombre,region_nombre,latitud,longitud,potencia_maxima
398,318,eolica-los-cururos,1.03,2016-09-18 03:00:00,PE LOS CURUROS,Eólicas,Coquimbo,-31.029949,-71.657108,109.4
399,318,eolica-los-cururos,0.51,2016-09-18 03:00:00,PE LOS CURUROS,Eólicas,Coquimbo,-31.029949,-71.657108,109.4
403,318,eolica-los-cururos,0.46,2016-09-18 04:00:00,PE LOS CURUROS,Eólicas,Coquimbo,-31.029949,-71.657108,109.4
404,318,eolica-los-cururos,0.48,2016-09-18 04:00:00,PE LOS CURUROS,Eólicas,Coquimbo,-31.029949,-71.657108,109.4
405,318,eolica-los-cururos,0.08,2016-09-18 05:00:00,PE LOS CURUROS,Eólicas,Coquimbo,-31.029949,-71.657108,109.4
...,...,...,...,...,...,...,...,...,...,...
1285999,649,csp-cerro-dominador-en,26.00,2021-04-20 06:00:00,CSP CERRO DOMINADOR,Termoeléctricas,Antofagasta,-22.776312,-69.480543,114.4
1286292,649,csp-cerro-dominador-en,80.00,2021-04-20 11:00:00,CSP CERRO DOMINADOR,Termoeléctricas,Antofagasta,-22.776312,-69.480543,114.4
1286332,649,csp-cerro-dominador-en,80.00,2021-04-20 11:00:00,CSP CERRO DOMINADOR,Termoeléctricas,Antofagasta,-22.776312,-69.480543,114.4
1785952,649,csp-cerro-dominador-en,7.63,2022-06-08 07:00:00,CSP CERRO DOMINADOR,Termoeléctricas,Antofagasta,-22.776312,-69.480543,114.4


In [65]:
df_merged.to_csv(r'../data/pronostico_erv/data_erv.csv', index=False)

In [66]:
df_grouped = df_merged.groupby(['datetime','tipo_central_nombre','region_nombre']).agg({'pronostico':'sum'}).reset_index()
df_grouped.head()



,datetime,tipo_central_nombre,region_nombre,pronostico
0,2016-08-23 18:00:00,Solares,Atacama,14.6
1,2016-08-24 13:00:00,Solares,Atacama,22.0
2,2016-08-24 14:00:00,Solares,Atacama,22.0
3,2016-08-24 15:00:00,Solares,Atacama,22.0
4,2016-08-24 16:00:00,Solares,Atacama,22.0


In [67]:
df_grouped

,datetime,tipo_central_nombre,region_nombre,pronostico
0,2016-08-23 18:00:00,Solares,Atacama,14.60
1,2016-08-24 13:00:00,Solares,Atacama,22.00
2,2016-08-24 14:00:00,Solares,Atacama,22.00
3,2016-08-24 15:00:00,Solares,Atacama,22.00
4,2016-08-24 16:00:00,Solares,Atacama,22.00
...,...,...,...,...
526094,2022-07-02 15:00:00,Solares,Antofagasta,40.21
526095,2022-07-02 15:00:00,Solares,Atacama,132.83
526096,2022-07-02 16:00:00,Solares,Antofagasta,33.80
526097,2022-07-02 16:00:00,Solares,Atacama,104.96


In [68]:
df_grouped.to_csv(r'../data/pronostico_erv/data_erv_aggregated.csv', index=False)